In [12]:
import requests
from datetime import datetime, timezone
from pprint import pprint

from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()

# Now you can access your API keys directly
NOTION_TOKEN = os.environ.get("NOTION_TOKEN")
DATABASE_ID = os.environ.get("NOTION_DATABASE_ID")

headers = {
    "Authorization": "Bearer " + NOTION_TOKEN,
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28",
}

def create_page(data: dict):
    create_url = "https://api.notion.com/v1/pages"

    payload = {"parent": {"database_id": DATABASE_ID}, "properties": data}

    res = requests.post(create_url, headers=headers, json=payload)
    # print(res.status_code)
    return res

def get_pages(num_pages=None):
    """
    If num_pages is None, get all pages, otherwise just the defined number.
    """
    url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"

    get_all = num_pages is None
    page_size = 100 if get_all else num_pages

    payload = {"page_size": page_size}
    response = requests.post(url, json=payload, headers=headers)

    data = response.json()

    # Comment this out to dump all data to a file
    # import json
    # with open('db.json', 'w', encoding='utf8') as f:
    #    json.dump(data, f, ensure_ascii=False, indent=4)

    results = data["results"]
    while data["has_more"] and get_all:
        payload = {"page_size": page_size, "start_cursor": data["next_cursor"]}
        url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"
        response = requests.post(url, json=payload, headers=headers)
        data = response.json()
        results.extend(data["results"])

    return results

print(get_pages())

[{'object': 'page', 'id': '24b94cc6-e4ba-80b4-9e94-c842734febe3', 'created_time': '2025-08-10T06:08:00.000Z', 'last_edited_time': '2025-08-10T06:09:00.000Z', 'created_by': {'object': 'user', 'id': '21e866be-1234-4086-b3ac-ab83d1fae095'}, 'last_edited_by': {'object': 'user', 'id': '21e866be-1234-4086-b3ac-ab83d1fae095'}, 'cover': None, 'icon': None, 'parent': {'type': 'database_id', 'database_id': '24b94cc6-e4ba-8082-92ce-c1821f5473ac'}, 'archived': False, 'in_trash': False, 'properties': {'Call Number': {'id': 'HK_m', 'type': 'number', 'number': 615.222}, 'Library Location': {'id': 'HWa%60', 'type': 'rich_text', 'rich_text': [{'type': 'text', 'text': {'content': 'Sunnyvale Public Library', 'link': None}, 'annotations': {'bold': False, 'italic': False, 'strikethrough': False, 'underline': False, 'code': False, 'color': 'default'}, 'plain_text': 'Sunnyvale Public Library', 'href': None}]}, 'Date': {'id': 'LgQI', 'type': 'date', 'date': {'start': '2025-08-09', 'end': None, 'time_zone': No

In [14]:
pages = get_pages()

for page in pages:
    props = page["properties"]

    data = {
        "ID": page["id"],
        "Title": props["Title"]["title"][0]["text"]["content"],
        "Status": props["Status"]["status"]["name"],
        "Call Number": props["Call Number"]["number"],
        "Location": props["Library Location"]["rich_text"][0]["text"]["content"],
        "Date": props["Date"]["date"]["start"]
    }

    pprint(data)  # pretty print dict

{'Call Number': 615.222,
 'Date': '2025-08-09',
 'ID': '24b94cc6-e4ba-80b4-9e94-c842734febe3',
 'Location': 'Sunnyvale Public Library',
 'Status': 'All Copies in Use',
 'Title': 'Maybe you should talk to somone'}
